> Attempt this only if you have attempted the Question 1 of Bonus Assignment. Otherwise do that first

# Question 1
## Use PCA

Sve olthis question on Kaggle! Submit your predictions in Kaggle.
Download the notebook and submit it separately along with the other questions.

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview

Use everything that you have learnt so far.

Analyse the data. Drop, scale or normalize features. Merge features if they correlated to make some meaningful features.<br> 
**Use PCA to reduce the dimensionality of the problem.**

Try out all the regression models. Use cross validation to find best parameters for each model.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
main_path = '/Users/praneethsai/Downloads'
train_path = '/train.csv' ; test_path = '/test.csv'

train_data = pd.read_csv(main_path + train_path)
test_data = pd.read_csv(main_path + test_path)

In [3]:
len(train_data)

1460

In [4]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
NULL = {}
for feature in train_data.columns:
    null_values = train_data[feature].isnull().sum()
    if null_values > len(train_data)//10:
         NULL[feature] = null_values
NULL

{'LotFrontage': 259,
 'Alley': 1369,
 'MasVnrType': 872,
 'FireplaceQu': 690,
 'PoolQC': 1453,
 'Fence': 1179,
 'MiscFeature': 1406}

In [6]:
# we shouldn't remove instances which has Nan values
# which might cause sparesity of data, So drop the columns which are
# causing more nan values (uncertain about the features which cannot be used un determining the label)

drop_columns = NULL.keys()
train_data.drop(columns=drop_columns, inplace=True)


In [7]:
# Drop the nan values in every column .
train_data.dropna(inplace=True)


In [8]:
len(train_data)

1338

In [9]:
ex_gd_class = ['PoolQc', 'GargeCond', 'GargeQual', 'FireplaceQu', 'KitchenQual','HeatingQc', 'BsmtCond', 'BsmtQual', 'ExterCond', 
               'ExterQual']
gd_av_class = ['BsmtExposure']
glq_alq_class = ['BsmtFinType1', 'BsmtFinType2']

for feature in NULL.keys():
    if feature in ex_gd_class:
        ex_gd_class.remove(feature)
    elif feature in gd_av_class:
        gd_av_class.remove(feature)
    elif feature in glq_alq_class:
        glq_alq_class.remove(feature)

In [10]:
train_data.GarageCond

0       TA
1       TA
2       TA
3       TA
4       TA
        ..
1455    TA
1456    TA
1457    TA
1458    TA
1459    TA
Name: GarageCond, Length: 1338, dtype: object

In [11]:
for feature in ex_gd_class:
    if feature in train_data.columns:
        train_data[feature] = train_data[feature].replace({'Ex':4, 'Gd':3, 'TA':2, 'Fa':1, 'Po':-1, 'NA':0})

In [12]:
for feature in gd_av_class:
    if feature in train_data.columns:
        train_data[feature] = train_data[feature].replace({'Gd':3, 'Av':2, 'Mn':1, 'No':-1, 'NA':0})

In [13]:
for feature in glq_alq_class:
    if feature in train_data.columns:
        train_data[feature] = train_data[feature].replace({'GLQ':4, 'ALQ':3, 'BLQ':2, 'Rec':1, 'LwQ':0, 'Unf':-1, 'NA':-2})

In [14]:
train_data.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [15]:
from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder()

for feature in train_data.columns:
    if (feature in ex_gd_class) or (feature  in gd_av_class) or (feature in glq_alq_class):
        continue
    if train_data[feature].dtype == 'object':
        train_data[feature] = lbl.fit_transform(train_data[feature])

In [16]:
train_data.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,3,8450,1,3,3,0,4,0,...,0,0,0,0,0,2,2008,8,4,208500
1,2,20,3,9600,1,3,3,0,2,0,...,0,0,0,0,0,5,2007,8,4,181500
2,3,60,3,11250,1,0,3,0,4,0,...,0,0,0,0,0,9,2008,8,4,223500
3,4,70,3,9550,1,0,3,0,0,0,...,272,0,0,0,0,2,2006,8,0,140000
4,5,60,3,14260,1,0,3,0,2,0,...,0,0,0,0,0,12,2008,8,4,250000


In [17]:
# Correlation with Sales Prices
correlation = train_data.corr()
(correlation[((correlation['SalePrice']) > 0.5) | ((correlation['SalePrice']) < -0.5)])['SalePrice']

OverallQual     0.783546
YearBuilt       0.504297
YearRemodAdd    0.501435
ExterQual       0.669504
BsmtQual        0.643452
TotalBsmtSF     0.602042
1stFlrSF        0.604714
GrLivArea       0.711706
FullBath        0.569313
KitchenQual     0.650226
TotRmsAbvGrd    0.551821
GarageFinish   -0.508232
GarageCars      0.640154
GarageArea      0.607535
SalePrice       1.000000
Name: SalePrice, dtype: float64

In [18]:
i = 0
for feature in train_data.columns:
    if train_data[feature].dtype == 'object':
        print(feature) ; i+=1
i

0

In [19]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
train_data_scaled = (std.fit_transform(train_data))

In [20]:
from sklearn.decomposition import PCA

principal = PCA(n_components=10)
train_data_projected = principal.fit_transform(train_data_scaled)


In [21]:
eigen_vectors = principal.components_

In [22]:
train_data_projected

array([[ 2.33827437, -0.10346365, -2.03662886, ..., -0.4106492 ,
        -1.68261423, -0.57326332],
       [-0.05278024, -1.48116761,  1.53948731, ..., -0.47307813,
         1.14936485,  0.08852889],
       [ 2.80019683, -0.1880469 , -1.47461908, ..., -0.80682405,
        -1.00185434, -0.18752858],
       ...,
       [ 1.40952198,  2.19864837,  0.14616157, ...,  1.10993791,
         3.11637192, -0.57962222],
       [-3.22532995, -2.84374875,  2.55337072, ...,  3.90705695,
        -1.26969627,  0.83033068],
       [-1.31691347, -1.92886139,  1.59034455, ...,  0.3655023 ,
        -0.85752662,  0.19195576]])

In [23]:
X = train_data_projected[:, :-1]
y = train_data_projected[:, -1]
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [24]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV

hyper_parameter_dict = {
    'C' : [i for i in range(11)],
    'kernel' : ['rbf', 'linear', 'poly'],
    'degree' : range(2, 4),
    'gamma' : [0.001, 0.01] + [i/10 for i in range(1, 3)]
}


In [25]:
svr = SVR()
search = RandomizedSearchCV(estimator=svr, param_distributions=hyper_parameter_dict, random_state=42, scoring='accuracy')
        

In [26]:
search.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
 

RandomizedSearchCV(estimator=SVR(),
                   param_distributions={'C': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                                        'degree': range(2, 4),
                                        'gamma': [0.001, 0.01, 0.1, 0.2],
                                        'kernel': ['rbf', 'linear', 'poly']},
                   random_state=42, scoring='accuracy')

In [27]:
search.best_params_

{'kernel': 'poly', 'gamma': 0.1, 'degree': 3, 'C': 2}

In [ ]:
# USING SUPPORT VECTOR REGRSSION
# ------------------------------

In [28]:

Svr = SVR(kernel='poly', gamma=0.1, degree=3, C=2)
Svr.fit(X_train, y_train)
y_predicted = Svr.predict(X_test)
print(f'MSE : {mean_squared_error(y_predicted, y_test)}')

MSE : 1.0807883672457395


In [ ]:
# USING LINEAR REGRESSION 
# -----------------------

In [29]:

from sklearn.linear_model import LinearRegression
lgt = LinearRegression()
lgt.fit(X_train, y_train)
y_predicted = lgt.predict(X_test)
print(f'MSE : {mean_squared_error(y_predicted, y_test)}')

MSE : 1.384798830699866


In [ ]:
# USING A SIMPLE NEURAL NETWORK
# --------------------------

In [31]:

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import SGD

MODEL = Sequential([
    Input(shape=X_train[0].shape),
    Dense(units=10, activation='relu'),
    Dense(units=1, activation='linear')
])

In [32]:
MODEL.compile(SGD(),loss=MeanSquaredError(), metrics=['accuracy'])


In [34]:
history = MODEL.fit(X_train, y_train, epochs=20)

Epoch 1/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 491us/step - accuracy: 0.0000e+00 - loss: 1.2695
Epoch 2/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 392us/step - accuracy: 0.0000e+00 - loss: 1.3808
Epoch 3/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 366us/step - accuracy: 0.0000e+00 - loss: 1.0878
Epoch 4/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 380us/step - accuracy: 0.0000e+00 - loss: 1.3434
Epoch 5/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 376us/step - accuracy: 0.0000e+00 - loss: 1.1382
Epoch 6/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 382us/step - accuracy: 0.0000e+00 - loss: 1.0660
Epoch 7/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 369us/step - accuracy: 0.0000e+00 - loss: 1.0819
Epoch 8/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - accuracy: 0.0000e+00 - loss: 1.1051
Epoch 9/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 405us/step - accuracy: 0.0000e+00 - loss: 1.1341
Epoch 10/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 382us/step - accuracy: 0.0000e+00 - loss: 1.1206
Epoch 11/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 383us/step - accuracy: 0.0000e+00 - loss: 1.1456
Epoch 12

In [35]:
y_predicted = MODEL.predict(X_test)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [36]:
print(f'MSE : {mean_squared_error(y_predicted, y_test)}')

MSE : 0.838361949119362
